In [21]:
# get an auth token using the steps here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token
# set it in this cell, then delete it to avoid accidentally committing it
authtoken = ''  

In [23]:
import os

resultsDownloadLocation = 'c:\\temp\\testResults'
if (not os.path.exists(resultsDownloadLocation)):
    os.makedirs(resultsDownloadLocation)


In [15]:
import requests

def getRuns():
    runsResponse = requests.get(
        "https://api.github.com/repos/microsoft/vscode-jupyter/actions/workflows/aggregate-test-results.yml/runs?per_page=100",
        headers={"Accept": "application/vnd.github+json"},
    )

    print(f"Found {len(runsResponse.json()['workflow_runs'])} runs")

    return runsResponse.json()["workflow_runs"]

runs = getRuns()

Found 30 runs


In [24]:
import zipfile
import json
import io

def getArtifactData(id):
    testResultsResponse = requests.get(
        f"https://api.github.com/repos/microsoft/vscode-jupyter/actions/artifacts/{id}/zip",
        headers={
            "Accept": "application/vnd.github+json",
            "Authorization": f"Bearer {authtoken}",
        },
    )

    if testResultsResponse.status_code != 200:
        print(f"Error {testResultsResponse.status_code} getting artifact {id}")

    return testResultsResponse.content
    
def saveResultsFile(zipData):
    with zipfile.ZipFile(io.BytesIO(zipData)) as artifact:
        for name in artifact.namelist():
            content = artifact.read(name)
            print(f"    saving {name}")
            with open(f'{resultsDownloadLocation}\\{name}', 'wb') as f:
                f.write(content)
        

print(f"Getting artifacts from {len(runs)} runs")
for run in runs:
    artifactUrl = run["artifacts_url"]
    print(f"Getting artifacts from {artifactUrl} from {run['created_at']}")
    artifactsResponse = requests.get(
        artifactUrl, headers={"Accept": "application/vnd.github+json"}
    )

    artifacts = artifactsResponse.json()["artifacts"]
    for artifact in artifacts:
        rawData = getArtifactData(artifact["id"])
        testRunResults = saveResultsFile(rawData)

Getting artifacts from 30 runs
Getting artifacts from https://api.github.com/repos/microsoft/vscode-jupyter/actions/runs/3040888074/artifacts from 2022-09-12T22:06:21Z
    saving AggTestResults-2022-09-01.json
Getting artifacts from https://api.github.com/repos/microsoft/vscode-jupyter/actions/runs/3040887031/artifacts from 2022-09-12T22:06:11Z
    saving AggTestResults-2022-09-02.json
Getting artifacts from https://api.github.com/repos/microsoft/vscode-jupyter/actions/runs/3040876099/artifacts from 2022-09-12T22:04:13Z
    saving AggTestResults-2022-09-04.json
Getting artifacts from https://api.github.com/repos/microsoft/vscode-jupyter/actions/runs/3040874992/artifacts from 2022-09-12T22:04:05Z
    saving AggTestResults-2022-09-05.json
Getting artifacts from https://api.github.com/repos/microsoft/vscode-jupyter/actions/runs/3040874261/artifacts from 2022-09-12T22:03:55Z
    saving AggTestResults-2022-09-06.json
Getting artifacts from https://api.github.com/repos/microsoft/vscode-jupyt